This notebook contains an example of using `redbiom` through it's Python API to extract a subset of American Gut Project samples. These data are then loaded into QIIME 2 for a mini beta-diversity analysis using UniFrac. This assumes we're using a QIIME 2 2018.11 environment that additionally has `redbiom` 0.3.0 installed. The exact commands I ran to install it are:

```
$ conda install nltk
$ pip install https://github.com/biocore/redbiom/archive/0.3.0.zip
```

In [3]:
import redbiom.summarize
import redbiom.search
import redbiom.fetch
import qiime2

The first thing we're going to do is gather the `redbiom` contexts. A context is roughly a set of consistent technical parameters. For example, the specific sequenced gene, the variable region within the gene, the length of the read, and how the operational taxonomic units were assessed.

The reason `redbiom` partitions data into contexts is because these technical details can lead to massive technical bias. The intention is to facilitate comparing "apples" to "apples". 

The context summarization returns a pandas `DataFrame` so it should be pretty friendly to manipulate.

In [4]:
contexts = redbiom.summarize.contexts()

In [5]:
contexts.head()

,ContextName,SamplesWithData,FeaturesWithData,Description
0,Pick_closed-reference_OTUs-Greengenes-ls454-16...,114,2868,Qiita context
1,Pick_closed-reference_OTUs-Greengenes-ls454-16...,3595,7921,Qiita context
2,Pick_closed-reference_OTUs-Greengenes-ls454-16...,50,331,Qiita context
3,Pick_closed-reference_OTUs-Greengenes-titanium...,339,9059,Qiita context
4,Pick_closed-reference_OTUs-Greengenes-ls454-16...,609,14847,Qiita context


At the present time, determining the context to use is a bit manual and requires some strung munging. Additional development is needed.

Let's take a look at the larger contexts.

In [6]:
contexts.sort_values('SamplesWithData', ascending=False).head()

,ContextName,SamplesWithData,FeaturesWithData,Description
38,Pick_closed-reference_OTUs-Greengenes-illumina...,129596,74983,Qiita context
98,Pick_closed-reference_OTUs-Greengenes-illumina...,128222,82492,Qiita context
25,Pick_closed-reference_OTUs-Greengenes-illumina...,125354,73083,Qiita context
73,Deblur-NA-illumina-16S-v4-100nt-fbc5b2,123127,5587560,Qiita context
7,Deblur-NA-illumina-16S-v4-90nt-99d1d8,119538,4460311,Qiita context


For simplicity sake, let's take the first context. It's large, and the phylogeny associated with the operational taxonomic units is easy to get. We'll break down the meaning of the context name in a moment. In practice, you will _most likely_ want to use the Deblur data, however producing a reasonable tree from those data requies a slightly computationally expensive step, and I'm on my laptop right now with limited battery quite literally in the middle of nowhere on a bus in the Czech Republic.

In [9]:
ctx = contexts.sort_values('SamplesWithData', ascending=False).iloc[0]['ContextName']

In [10]:
ctx

'Pick_closed-reference_OTUs-Greengenes-illumina-16S-v4-100nt-a243a1'

Breaking this name into its constiuent pieces, this is a closed reference context meaning that operational taxonomic units were assessed against a reference database and sequences which did not recruit to the reference were discarded. The reference used is Greengenes, a common 16S reference database. The gene represented by the data is the 16S SSU rRNA gene, and specifically the V4 region of the gene. Finally, the fragments represented are truncated to 100 nucleotides. (Don't worry if this is all a lot of jargon. It is a lot of jargon. Please ask questions :)

So cool, we have a "context". What can we do now? Let's search for some sample identifiers based off of the metadata (i.e., variables) associated with the samples. Specifically, let's get some skin, oral and fecal samples. Be forewarned, the metadata search uses Python's `ast` module behind the scenes, so malformed queries at present produce tracebacks.

In [18]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317
results = {}
for site in ['sebum', 'saliva', 'feces']:
    query = "where qiita_study_id==%d and env_material=='%s'" % (study_id, site)
    results[site] = redbiom.search.metadata_full(query)

In [19]:
for k, v in results.items():
    print(k, len(v))

feces 16207
saliva 1257
sebum 1136


As you can see, the sample sets are not very balanced. For the purposes of the mini-analysis, let's just grab the first 100 from each sample type.

In [20]:
to_keep = []
for k, v in results.items():
    to_keep.extend(list(v)[:100])

In [21]:
to_keep[:5]

['10317.000068373',
 '10317.000069792',
 '10317.000039986',
 '10317.000063077',
 '10317.000020031']

The last output cell shows what these IDs look like. These are Qiita sample IDs.

Now that we have some samples, let's get some data! What we're going to do is ask `redbiom` to obtain the sample data, for our `to_keep` samples, from the context we previously selected. What's happening behind the scenes is that the API is pulling out sparse vectors corresponding to the number of individual sequences observed for each operational taxonomic unit per sample, and additionally unmunging the names (as `redbiom` normalizes sample and feature identifiers). The output is then aggregated into what's called a BIOM `Table`, which is really just a rich object wrapped around a `scipy.sparse` matrix. 

You may noice two outputs on the return. The one we're ignoring represents "ambiguous" samples. Some sample identifiers are associated with multiple sequenced samples. This is because some samples may "fail" sequencing, where they didn't yield sufficient sequence data, and were rerun. These "failures" are still represented in Qiita, but are differentiated by the actual sequencing run they were on. This doesn't matter for the moment though.

In [25]:
biom_table, _ = redbiom.fetch.data_from_samples(ctx, to_keep)

In [26]:
biom_table

13509 x 244 <class 'biom.table.Table'> with 109450 nonzero entries (3% dense)

The `repr` output shows that we have roughly 13k OTUs (operational taxonomic units), and only 244 samples. What gives? We were supposed to get 300! Just because a sample has metadata does not mean it has sequence data. It is also possible that some of the samples haven't been run through the same bioinformatic processing (e.g., closed reference at 100nt).

More information on `biom` can be found [here](http://biom-format.org/). 

Let's play with the object for just a moment for familiarity.

In [27]:
biom_table.head()

5 x 5 <class 'biom.table.Table'> with 1 nonzero entries (4% dense)

In [28]:
print(biom_table.head())

# Constructed from biom file
#OTU ID	10317.000007200.48059	10317.000004286.47973	10317.000009072.47981	10317.000009081.47989	10317.000007370.48059
4410270	0.0	0.0	0.0	0.0	0.0
1126194	0.0	0.0	0.0	0.0	0.0
4331360	0.0	108.0	0.0	0.0	0.0
1043665	0.0	0.0	0.0	0.0	0.0
353085	0.0	0.0	0.0	0.0	0.0


When we print the table, it implicitly casts it to a dense matrix representation. One thing to note: the sample IDs look different than before, right? The way that the individual sequencing runs are denoted is the last number -- the sample ID is represented by "qiita_study_id.the_actual_id.the_sequencing_run_id". This is to ensure the identifier is "globally" unique.

The identifiers on the rows are the "Greengenes" IDs. Also called OTU IDs. Or Feature IDs.

So we have a table. Now let's get the sample metadata. This will come down as a pandas `DataFrame`.

In [30]:
md, _ = redbiom.fetch.sample_metadata(to_keep, context=ctx)

In [32]:
md.head()

,#SampleID,alcohol_types_beercider,alcohol_types_red_wine,alcohol_types_sour_beers,alcohol_types_spiritshard_alcohol,alcohol_types_unspecified,alcohol_types_white_wine,allergic_to_i_have_no_food_allergies_that_i_know_of,allergic_to_other,allergic_to_peanuts,...,subset_age,subset_antibiotic_history,subset_bmi,subset_diabetes,subset_healthy,subset_ibd,survey_id,taxon_id,title,weight_units
47970_10317.000007776,10317.000007776.47970,false,false,false,false,true,false,false,false,false,...,false,false,false,true,false,true,63858372b3359691,539655,American Gut Project,kilograms
47970_10317.000007831,10317.000007831.47970,false,false,false,false,true,false,false,false,false,...,true,true,false,true,false,true,9aa4aa6f19982163,539655,American Gut Project,kilograms
47970_10317.000007835,10317.000007835.47970,false,false,false,false,true,false,false,false,false,...,false,false,false,true,false,true,63858372b3359691,447426,American Gut Project,kilograms
47970_10317.000007838,10317.000007838.47970,false,false,false,false,true,false,false,false,false,...,false,false,false,true,false,true,63858372b3359691,539655,American Gut Project,kilograms
47970_10317.000007847,10317.000007847.47970,false,false,false,false,true,false,false,false,false,...,true,true,false,true,false,true,9aa4aa6f19982163,539655,American Gut Project,kilograms


In [33]:
md.sample_type.value_counts()

Mouth         78
Stool         75
Right Hand    31
Forehead      28
Left Hand     20
Torso         10
Left leg       1
Hair           1
Name: sample_type, dtype: int64

Great! So we have a bunch of sample data, and some study variables! Now for the QIIMEing. What we'll need to do is pacakge these data into types that QIIME 2 understands. And in particular, we need to use the semantic type system. This is well documented on the QIIME 2 website, and I recommend reviewing there.

In [35]:
table_ar = qiime2.Artifact.import_data('FeatureTable[Frequency]', biom_table)
md_ar = qiime2.Metadata(md.set_index('#SampleID'))

Ah! We need a tree! Since we're using Greengenes, we can just rely on the existing prebuilt tree from the reference. Let's get that.

In [36]:
!wget ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/trees/97_otus.tree

--2019-01-07 02:25:28--  ftp://ftp.microbio.me/greengenes_release/gg_13_8_otus/trees/97_otus.tree
           => '97_otus.tree'
Resolving ftp.microbio.me... 169.228.46.98
Connecting to ftp.microbio.me|169.228.46.98|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /greengenes_release/gg_13_8_otus/trees ... done.
==> SIZE 97_otus.tree ... 2542738
==> PASV ... done.    ==> RETR 97_otus.tree ... done.
Length: 2542738 (2.4M) (unauthoritative)

97_otus.tree        100%[===================>]   2.42M   172KB/s    in 17s     

2019-01-07 02:25:49 (150 KB/s) - '97_otus.tree' saved [2542738]



In [37]:
tree_ar = qiime2.Artifact.import_data('Phylogeny[Rooted]', '97_otus.tree')

Excellent. So next up, we need to rarefy these data. Rarefaction is a just randomly subsampling the samples without replacement in order to normalize for sequencing effort. There are a lot of ways to do this type of normalization. Rarefaction is dirty as you through out data, but it's pragmatic and tends to work pretty well. The issue rarefaction helps to ameliorate is that, the more you sequence the more life you'll observe. So if you don't do something to normalize sequencing effort across samples, you'll have a bad time.

Let's import all the plugins we'll be using.

In [40]:
from qiime2.plugins import feature_table, diversity, emperor

In [45]:
# rarefy to 1000 sequences per sample (yes, it's arbitrary)
rare_ar, = feature_table.actions.rarefy(table=table_ar, sampling_depth=1000)

In [46]:
unweighted_unifrac_ar, = diversity.actions.beta_phylogenetic(table=rare_ar, 
                                                             phylogeny=tree_ar, 
                                                             metric='unweighted_unifrac')

So before we move on, we should discuss what this voodoo just did. Beta diversity is what ecologists call comparing how similar (or dissimilar) two samples are. For example, how similar is a forest in the Pacific Northwest to a forest in Maine? (i.e., beta diversity is a function which takes two vectors and returns a scaler). We perform this calculation over all pairs of samples though so that we can examine all of the sample relationships. This distance matrix gets large quick: in our case, it's already 244 by 244. 

Visualizing that matrix would suck. So, one thing we often do is principal *coordinates* analysis. It's very similar to principal *components* analysis, except that we can pass in our distance matrix.

In [47]:
unweighted_unifrac_pcoa_ar, = diversity.actions.pcoa(unweighted_unifrac_ar)

/Users/dtmcdonald/miniconda3/envs/qiime2-2018.11/lib/python3.5/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.023874671034204868 and the largest is 13.456602547170576.
  RuntimeWarning


Finally, let's actually view the coordinates. It's much more interesting to use the metadata as well :) Hint: color by body site.

In [50]:
viz, = emperor.actions.plot(unweighted_unifrac_pcoa_ar, md_ar)

In [51]:
viz

<visualization: Visualization uuid: d5ea7621-7d93-403e-afe2-1fd118b89507>